In [1]:
# 本代码由可视化策略环境自动生成 2019年7月27日 16:00
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5

# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    #获取当日日期
    today = data.current_dt.strftime('%Y-%m-%d')
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions ]
    #大盘风控模块，读取风控数据    
    benckmark_risk=context.benckmark_risk[today]

    #当risk为1时，市场有风险，全部平仓，不再执行其它操作
    if benckmark_risk > 0:
        for instrument in stock_hold_now:
            context.order_target(symbol(instrument), 0)
        print(today,'大盘风控止损触发,全仓卖出')
        return
 
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.portfolio.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities)])))
        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)
# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    #在数据准备函数中一次性计算每日的大盘风控条件相比于在handle中每日计算风控条件可以提高回测速度
    # 多取50天的数据便于计算均值(保证回测的第一天均值不为Nan值)，其中context.start_date和context.end_date是回测指定的起始时间和终止时间
    start_date= (pd.to_datetime(context.start_date) - datetime.timedelta(days=50)).strftime('%Y-%m-%d') 
    df=DataSource('bar1d_index_CN_STOCK_A').read(start_date=start_date,end_date=context.end_date,fields=['close'])

    #这里以上证指数000001.HIX为例
    benckmark_data=df[df.instrument=='000001.HIX']
    #计算上证指数5日涨幅
    benckmark_data['ret5']=benckmark_data['close']/benckmark_data['close'].shift(5)-1
    #计算大盘风控条件，如果5日涨幅小于-4%则设置风险状态risk为1，否则为0
    benckmark_data['risk'] = np.where(benckmark_data['ret5']<-0.04,1,0)
    #修改日期格式为字符串(便于在handle中使用字符串日期索引来查看每日的风险状态)
    benckmark_data['date']=benckmark_data['date'].apply(lambda x:x.strftime('%Y-%m-%d'))
    #设置日期为索引
    benckmark_data.set_index('date',inplace=True)
    #把风控序列输出给全局变量context.benckmark_risk
    context.benckmark_risk=benckmark_data['risk']

m1 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2015-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
return_5
return_10
return_20
avg_amount_0/avg_amount_5
avg_amount_5/avg_amount_20
rank_avg_amount_0/rank_avg_amount_5
rank_avg_amount_5/rank_avg_amount_10
rank_return_0
rank_return_5
rank_return_10
rank_return_0/rank_return_5
rank_return_5/rank_return_10
pe_ttm_0
"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m13 = M.dropnan.v1(
    input_data=m7.data
)

m6 = M.stock_ranker_train.v5(
    training_ds=m13.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    m_lazy_run=False
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2015-01-01'),
    end_date=T.live_run_param('trading_date', '2017-01-01'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m14 = M.dropnan.v1(
    input_data=m18.data
)

m8 = M.stock_ranker_predict.v5(
    model=m6.model,
    data=m14.data,
    m_lazy_run=False
)

m19 = M.trade.v4(
    instruments=m9.data,
    options_data=m8.predictions,
    start_date='',
    end_date='',
    initialize=m19_initialize_bigquant_run,
    handle_data=m19_handle_data_bigquant_run,
    prepare=m19_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2019-04-03 13:12:15.938405] INFO: bigquant: instruments.v2 开始运行..

[2019-04-03 13:12:15.962200] INFO: bigquant: 命中缓存

[2019-04-03 13:12:15.964034] INFO: bigquant: instruments.v2 运行完成[0.025643s].

[2019-04-03 13:12:15.968965] INFO: bigquant: advanced_auto_labeler.v2 开始运行..

[2019-04-03 13:12:15.975201] INFO: bigquant: 命中缓存

...

2015-01-14 大盘风控止损触发,全仓卖出
2015-01-30 大盘风控止损触发,全仓卖出
2015-02-02 大盘风控止损触发,全仓卖出
2015-02-03 大盘风控止损触发,全仓卖出
2015-02-06 大盘风控止损触发,全仓卖出
2015-05-05 大盘风控止损触发,全仓卖出
2015-05-06 大盘风控止损触发,全仓卖出
2015-05-07 大盘风控止损触发,全仓卖出
2015-05-08 大盘风控止损触发,全仓卖出
2015-06-16 大盘风控止损触发,全仓卖出
2015-06-18 大盘风控止损触发,全仓卖出
2015-06-19 大盘风控止损触发,全仓卖出
2015-06-23 大盘风控止损触发,全仓卖出
2015-06-24 大盘风控止损触发,全仓卖出
2015-06-25 大盘风控止损触发,全仓卖出
2015-06-26 大盘风控止损触发,全仓卖出
2015-06-29 大盘风控止损触发,全仓卖出
2015-06-30 大盘风控止损触发,全仓卖出
2015-07-01 大盘风控止损触发,全仓卖出
2015-07-02 大盘风控止损触发,全仓卖出
2015-07-03 大盘风控止损触发,全仓卖出
2015-07-06 大盘风控止损触发,全仓卖出
2015-07-07 大盘风控止损触发,全仓卖出
2015-07-08 大盘风控止损触发,全仓卖出
2015-07-09 大盘风控止损触发,全仓卖出
2015-07-27 大盘风控止损触发,全仓卖出
2015-07-28 大盘风控止损触发,全仓卖出
2015-07-29 大盘风控止损触发,全仓卖出
2015-07-30 大盘风控止损触发,全仓卖出
2015-07-31 大盘风控止损触发,全仓卖出
2015-08-18 大盘风控止损触发,全仓卖出
2015-08-20 大盘风控止损触发,全仓卖出
2015-08-21 大盘风控止损触发,全仓卖出
2015-08-24 大盘风控止损触发,全仓卖出
2015-08-25 大盘风控止损触发,全仓卖出
2015-08-26 大盘风控止损触发,全仓卖出
2015-08-27 大盘风控止损触发,全仓卖出
2015-08-28 大盘风控止损触发,全仓卖出
2015-09-15 大盘风控止损触发,全仓卖出
2015-09-29 大盘风控止损触发,全仓卖出


[2019-04-03 13:13:11.785769] INFO: Performance: Simulated 488 trading days out of 488.

[2019-04-03 13:13:11.787829] INFO: Performance: first open: 2015-01-05 09:30:00+00:00

[2019-04-03 13:13:11.789419] INFO: Performance: last close: 2016-12-30 15:00:00+00:00

[2019-04-03 13:13:15.006837] INFO: bigquant: backtest.v8 运行完成[58.858781s].